# n-Photon Compton Scattering Process

In this file, we set up an n-photon Compton scattering process. A Compton scattering
process looks like $k^n e^- \to k e^-$.

You can download this file as a [jupyter notebook](compton.ipynb).

In [1]:
using QEDFeynmanDiagrams

We need some of the packages of the [QEDjl-project](https://github.com/QEDjl-project) for base
functionality and the `ScatteringProcess` type.

In [2]:
using QEDcore
using QEDprocesses

Let's decide how many photons our electron interacts with:

In [3]:
n = 4;

Now we setup the scattering process accordingly. We consider all spin/polarization
combinations of the particles except for the incoming photons.

!!! note
    The functionality is not quite ready yet, but in the future, the incoming photons
    can be synced using `SyncedPol` from QEDbase. This would emulate all incoming
    photons having the same polarization, for example from a laser.

In [4]:
proc = ScatteringProcess(
    (Electron(), ntuple(_ -> Photon(), n)...),  # incoming particles
    (Electron(), Photon()),                     # outgoing particles
    (AllSpin(), ntuple(_ -> PolX(), n)...),     # incoming particle spin/pols
    (AllSpin(), AllPol()),                      # outgoing particle spin/pols
)

generic QED process
    incoming: electron (all spins), photon (x-polarized), photon (x-polarized), photon (x-polarized), photon (x-polarized)
    outgoing: electron (all spins), photon (all polarizations)


The `feynman_diagrams` function returns an iterator for all possible Feynman diagrams
for this scattering process. With its `length` overload, we can check how many diagrams
there are. For an n-photon Compton process with `n` incoming photons, this should be
$(n+1)!$.

In [5]:
length(feynman_diagrams(proc))

120

Next, we can generate the DAG representing the computation for our scattering process'
squared matrix element. This uses [`ComputableDAGs.jl`](https://github.com/ComputableDAGs/ComputableDAGs.jl).

In [6]:
dag = generate_DAG(proc)

Graph:
  Nodes: Total: 1210, QEDFeynmanDiagrams.ComputeTask_Pair: 136, QEDFeynmanDiagrams.ComputeTask_Propagator: 30, 
         QEDFeynmanDiagrams.ComputeTask_CollectPairs: 80, ComputableDAGs.DataTask: 625, QEDFeynmanDiagrams.ComputeTask_Triple: 240, 
         QEDFeynmanDiagrams.ComputeTask_BaseState: 10, QEDFeynmanDiagrams.ComputeTask_CollectTriples: 8, QEDFeynmanDiagrams.ComputeTask_PropagatePairs: 80, 
         QEDFeynmanDiagrams.ComputeTask_SpinPolCumulation: 1
  Edges: 2161
  Total Compute Effort: 0.0
  Total Data Transfer: 0.0
  Total Compute Intensity: 0.0


In this graph output you can see the number of nodes necessary to compute.
Note that for larger processes, the number of total nodes can be *lower* than
the number of Feynman diagrams, even with the added complexity of considering
multiple spin and polarization combinations. This is the result of efficient
reuse of reappearing parts of Feynman diagrams.

To continue, we will need [`ComputableDAGs.jl`](https://github.com/ComputableDAGs/ComputableDAGs.jl). Since `ComputableDAGs.jl` uses
`RuntimeGeneratedFunction`s as the return type of `ComputableDAGs.get_compute_function`, we need
to initialize it in our current module.

In [7]:
using ComputableDAGs
using RuntimeGeneratedFunctions
RuntimeGeneratedFunctions.init(@__MODULE__)

With the DAG, the process, and `RuntimeGeneratedFunctions` initalized,
we can now generate the actual computable function:

In [8]:
func = get_compute_function(dag, proc, cpu_st(), @__MODULE__);

Now we need an input for the function, which is a `QEDcore.PhaseSpacePoint`.
For now, we generate random momenta for every particle. In the future, QEDevents
will be able to generate physical `PhaseSpacePoint`s.

In [9]:
psp = PhaseSpacePoint(
    proc,
    PerturbativeQED(),
    PhasespaceDefinition(SphericalCoordinateSystem(), ElectronRestFrame()),
    tuple((rand(SFourMomentum) for _ in 1:number_incoming_particles(proc))...),
    tuple((rand(SFourMomentum) for _ in 1:number_outgoing_particles(proc))...),
)

PhaseSpacePoint:
    process: generic QED process "ekkkk -> ek"
    model: perturbative QED
    phasespace definition: spherical coordinates in electron rest frame
    incoming particles:
     -> incoming electron: [0.07871027126959518, 0.7491719221293475, 0.9079183159493375, 0.3265147419429981]
     -> incoming photon: [0.6070508637289256, 0.6548584614838039, 0.22879409879709556, 0.7745775266541413]
     -> incoming photon: [0.4173347911798654, 0.9108705762491486, 0.16840581233460983, 0.7902870783958265]
     -> incoming photon: [0.7397847988998845, 0.7295983277950986, 0.5007041519961887, 0.6057052667802503]
     -> incoming photon: [0.2770421820241452, 0.20461658150460194, 0.005321660723108801, 0.301926736513905]
    outgoing particles:
     -> outgoing electron: [0.6473406603816454, 0.522671823301556, 0.37087997178488263, 0.45732694139696806]
     -> outgoing photon: [0.07067201607487783, 0.23174149287647405, 0.908861232283568, 0.897713309130109]


Finally, we can test that the function actually runs and computes something by
simply calling it on the `PhaseSpacePoint`:

In [10]:
func(psp)

0.0049110553759556105

If we want, we can benchmark the execution speed too:

In [11]:
using BenchmarkTools
@benchmark func($psp)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  22.562 μs … 68.767 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     23.023 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   23.592 μs ±  2.165 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▄█▇▆▅▄▃▂▂▁▁                                                 ▂
  ████████████████▇▇▆▇▆▇▆▆▅▆▅▅▅▄▄▄▄▅▄▅▅▄▅▄▃▄▄▃▂▄▄▂▂▅▅▆▇█▇▇▇▆▆ █
  22.6 μs      Histogram: log(frequency) by time      33.6 μs <

 Memory estimate: 2.69 KiB, allocs estimate: 26.

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*